In [1]:
import os
from dotenv import load_dotenv
import tempfile
from chromadb import Client
from chromadb.config import Settings

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import MultiRetrievalQAChain
from langchain.memory import ConversationBufferMemory

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
#오픈AI API 키 설정
load_dotenv()  # 현재 경로의 .env 로드
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
if not os.environ['OPENAI_API_KEY']:
    raise ValueError('OPENAI_API_KEY not found in environment. set it in .env or env vars')

In [3]:
file_path = '../data/langchainThon/cv/경력기술서_천승우.docx'
# 파일을 바이너리 모드로 읽어서 _file처럼 흉내내기
class DummyFile:
    def __init__(self, path):
        self.name = path
        with open(path, "rb") as f:
            self._data = f.read()
    def getvalue(self):
        return self._data

_file = DummyFile(file_path)

In [4]:
# CV 불러오기 (PDF/Word)
def load_cv(_file):
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as tmp_file:
        tmp_file.write(_file.getvalue())
        tmp_file_path = tmp_file.name
        
    #PDF 파일 업로드
    if _file.name.endswith('.pdf'):
        loader = PyPDFLoader(file_path=tmp_file_path)
    
    #Word 파일 업로드
    elif _file.name.endswith('.docx'):
        loader = Docx2txtLoader(file_path=tmp_file_path)
    
    # 파일 형식이 틀릴경우 에러 메세지 출력
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 업로드해주세요.")
    
    pages = loader.load()
    return pages

# JD 불러오기 (URL)
def load_jd(_url: str):
    import warnings
    from urllib3.exceptions import InsecureRequestWarning
    warnings.simplefilter('ignore', InsecureRequestWarning)
    
    loader = WebBaseLoader(_url)
    loader.requests_kwargs = {'verify': False}  # SSL 검증 비활성화
    return loader.load()

In [5]:
def chunk_documents(docs, source_label, chunk_size=4000, chunk_overlap=100):
    '''
    페이지(Document list)들을 청크로 나누고 metadata를 추가하는 함수
    - docs: Document list
    - source_label: 'cv' 또는 'jd' 등 source 표시
    - return: 청크가 나뉜 Document 리스트
    '''
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', ' '],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )

    split_docs = text_splitter.split_documents(docs)

    split_docs_with_meta = [
        Document(page_content=d.page_content,
                 metadata={'source': source_label, 'chunk_id': str(i)},)
        for i, d in enumerate(split_docs)
    ]

    return split_docs_with_meta

In [6]:
cv_docs = chunk_documents(load_cv(_file), 'cv')

In [7]:
jd_docs = chunk_documents(load_jd('https://www.wanted.co.kr/wd/297589'), 'jd')

In [8]:
# 클라이언트 및 임베딩 모델 초기화
client_chroma = Client(Settings(is_persistent=False))
embeddings  = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=cv_docs+jd_docs,
    embedding=embeddings,
    client=client_chroma,
)

In [ ]:
# -----------------------------
# Retriever 설정
# -----------------------------
# CV Retriever
cv_retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'filter': {'source': 'cv'}}
)
# JD Retriever
jd_retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'filter': {'source': 'jd'}}
)
# Default Retriever
default_retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 6}
)

# -----------------------------
# System prompt 생성
# -----------------------------
system_prompt = '''
당신은 커리어 및 채용 관련 질의응답을 돕는 AI 어시스턴트입니다.  
아래 제공된 문서들을 참고하여, 사용자의 질문에 정확하고 논리적으로 답변하세요.  

- 사용자의 **경력, 경험, 이력 관련 질문**이라면 CV(이력서)를 우선적으로 참고하세요.  
- **지원하는 회사, 직무, 채용 요건** 관련 질문이라면 JD(공고문)를 우선적으로 참고하세요.  
- 질문의 맥락상 두 문서가 모두 관련되어 있다면, **균형 있게 통합하여 답변**하세요.  
- 문서에 직접적인 정보가 없을 경우, 일반적인 HR/커리어 상식과 논리를 기반으로 보완해 설명할 수 있습니다.
- 자기소개서 작성 또는 면접 질문 생성 요청시 각각 아래와 같은 지침을 따릅니다.

** 자기소개서 작성 지침**
- 자연스럽고 억지스럽지 않은 문장으로 꼭 필요한 부분에만 CV 내용을 활용합니다.
- 경험과 관련된 사례를 들어야 할 경우 반드시 CV에 기술된 내용만을 사실대로 말합니다.
- 작성된 경험들을 적절하게 JD에서 요구하는 능력 및 취업 후 성과 기여가 가능함의 근거로써 활용합니다.
- 자기소개서 작성시 글자수 제한은 한글을 기준으로 띄어쓰기를 포함하여 계산합니다.
- 질문지에 질문의 항목 또는 번호가 있더라도, 문장 형태로 답을 작성합니다.

** 면접 질문 생성 지침**
- 회사 JD를 참고하여 지원자의 능력을 평가할 수 있을만한 질문을 생성합니다.
- 일반적인 HR/커리어 상식과 논리를 기반으로 면접 질문을 생성 할 수 있습니다.

**언어 스타일 지침**
- 답변은 언제나 **전문적이고 신뢰감 있는 어투**로 작성합니다.  
- 문장은 명료하고 간결하게 유지하되, 비즈니스 상황에 맞는 적절한 어휘를 사용합니다.  
- 필요 시 가벼운 이모지를 활용하여 자연스럽게 친근함을 더할 수 있으나, 과도하게 사용하지 않습니다.  
    
문서 내용:
{context}
'''

# -----------------------------
# ChatPromptTemplate 생성
# -----------------------------
qa_prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', '{question}')
])


# -----------------------------
# MultiRetrievalQAChain 구성
# -----------------------------
retriever_infos = [
    {
        'name': 'cv',
        'description': 'user의 경력(CV) 및 이전 회사 관련 질문',
        'retriever': cv_retriever,
        'prompt': qa_prompt
    },
    {
        'name': 'jd',
        'description': '지원하는 회사 및 직무기술서(JD) 관련 질문',
        'retriever': jd_retriever,
        'prompt': qa_prompt
    },
]

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)

rag_chain = MultiRetrievalQAChain.from_retrievers(
    llm=llm,
    retriever_infos=retriever_infos,
    default_retriever=default_retriever,
)

In [10]:
query = '내가 다녔던 회사 이름과 근무 기간을 맞춰봐. 추가로 이전 회사에서 했던 주요 프로젝트 네가지 간단히 요약해줘.'
response = rag_chain.invoke({'input': query})

# LLM 답변
print('🧠 GPT-4o-mini 답변:')
print(response['result'])  # MultiRetrievalQAChain는 output_keys=['result']

🧠 GPT-4o-mini 답변:
귀하가 다녔던 회사는 (주)온코크로스이며, 근무 기간은 2021년 1월부터 2024년 12월까지 총 4년입니다. 

이전 회사에서 수행한 주요 프로젝트 네 가지는 다음과 같습니다:

1. **항암제 신규 적응증 탐색 플랫폼 개발 프로젝트**: 플랫폼 전체 기능을 Python으로 재구현하고, 23개 암종에 걸쳐 약 24,000개 샘플 데이터를 수집하여 대규모 분석 기반을 구축했습니다. 이 프로젝트를 통해 중국 제약사의 신규 항암제 적응증 예측에 성공했습니다.

2. **항암제 플랫폼 고도화 프로젝트**: 암종 별 유전자 예후 예측 스코어링 알고리즘을 고도화하여 알고리즘 스코어링 속도를 1100% 향상시켰습니다. 이를 통해 기존 알고리즘의 소요 시간을 크게 단축했습니다.

3. **엑체생검을 통한 암 조기 진단 모델 개발**: cfDNA 및 ctDNA 데이터를 수집하고 분석하여 암 여부 예측 딥러닝 모델을 개발했습니다. 이 모델은 Recall 0.69, Precision 0.64, ROC-AUC 0.71의 성과를 기록했습니다.

4. **신규 COVID-19 치료 후보 약물 탐색 및 연구 결과 논문 게재**: Dry-lab 연구를 통해 COVID-19 치료 유력 후보물질을 발굴하고, 연구 결과를 Scientific Reports 저널에 제출했습니다. 현재까지 논문 수정 작업이 진행 중입니다.


In [11]:
query = '내가 지원하려고하는 회사는?'
response = rag_chain.invoke({'input': query})

# LLM 답변
print('🧠 GPT-4o-mini 답변:')
print(response['result'])  # MultiRetrievalQAChain는 output_keys=['result']

🧠 GPT-4o-mini 답변:
당신이 지원하려고 하는 회사는 **임팩티브AI**입니다. 이 회사는 수요예측에 특화된 AI 솔루션 전문기업으로, 자체 솔루션인 **Deepflow**를 통해 제품 수요예측, 원자재 가격예측, 신제품 성과예측 등을 수행하고 있습니다. 임팩티브AI는 제조 및 유통 산업에서 기술력을 인정받으며 고객사의 성과 창출을 지원하고 있습니다.


In [12]:
query = '지원하신 직무에서 성과를 창출하기 위한 본인만의 핵심역량을 제시하고 이것이 (1)직무전문성, (2)조직기여 측면에서 어떻게 발휘될 수 있는지 사례를 들어 설명하시오. (최대 800자 입력가능)'
response = rag_chain.invoke({'input': query})

# LLM 답변
print('🧠 GPT-4o-mini 답변:')
print(response['result'])  # MultiRetrievalQAChain는 output_keys=['result']

🧠 GPT-4o-mini 답변:
제가 지원한 데이터 사이언티스트 직무에서 성과를 창출하기 위한 핵심역량은 **데이터 분석 및 머신러닝 모델링 능력**입니다. 이 역량은 (1) 직무전문성 측면에서, 시계열 예측 모델 및 시스템 구축 프로젝트 경험을 통해 강화되었습니다. 예를 들어, 이전 직장에서 고객의 판매 데이터를 분석하여 머신러닝 모델을 구축한 경험이 있습니다. 이를 통해 6개월 간의 판매량을 20% 정확도로 예측할 수 있었고, 이 결과는 고객사의 재고 관리 최적화에 기여하였습니다. 이러한 경험은 임팩티브AI의 Deepflow 솔루션을 통해 수요예측 및 재고관리 최적화에 직접적으로 기여할 수 있는 기반이 됩니다.

(2) 조직기여 측면에서는, 팀 내 소프트웨어 엔지니어 및 도메인 전문가와의 협업을 통해 예측모델을 개발하고 최적화하는 과정에서 제 역량이 발휘될 수 있습니다. 예를 들어, 데이터 전처리 및 피처 엔지니어링 과정에서 팀원들과의 원활한 소통과 협력을 통해 모델의 성능을 극대화할 수 있었습니다. 이러한 협업 경험은 임팩티브AI의 목표인 고객사의 성과 창출을 지원하는 데 중요한 역할을 할 것입니다. 

결론적으로, 저의 데이터 분석 및 머신러닝 모델링 능력은 직무전문성과 조직기여 두 측면에서 임팩티브AI의 비즈니스 목표 달성에 기여할 수 있는 강력한 자산이 될 것입니다.


In [15]:
query = '스스로 지원하려고 하는 회사의 면접관이 되어 면접에서 할 수 있는 질문 3가지를 출력해줘.'
response = rag_chain.invoke({'input': query})

# LLM 답변
print('🧠 GPT-4o-mini 답변:')
print(response['result'])  # MultiRetrievalQAChain는 output_keys=['result']

🧠 GPT-4o-mini 답변:
면접관으로서 지원자에게 물어볼 수 있는 질문은 다음과 같습니다:

1. 시계열 예측 모델을 구축한 경험에 대해 말씀해 주시겠습니까? 해당 프로젝트에서 사용한 데이터와 모델링 기법, 그리고 그 결과가 어떻게 고객에게 가치를 제공했는지 구체적으로 설명해 주세요.

2. 데이터 전처리 및 피처 엔지니어링 과정에서 직면했던 도전 과제는 무엇이었으며, 이를 어떻게 해결하였는지에 대해 이야기해 주실 수 있나요? 이 과정에서 어떤 도구나 기술을 사용하셨는지도 포함해 주세요.

3. 최신 머신러닝 및 데이터 분석 트렌드 중 하나를 선택하여, 이를 임팩티브AI의 예측 모델에 어떻게 적용할 수 있을지에 대한 귀하의 의견을 말씀해 주세요. 이 기술이 회사의 비즈니스에 어떤 긍정적인 영향을 미칠 수 있을지에 대해서도 설명해 주시면 좋겠습니다.
